In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
testdata=pd.read_csv("/Users/MacAir/Documents/DataScience/Datasets/home-data-for-ml-course/test.csv")
traindata=pd.read_csv("/Users/MacAir/Documents/DataScience/Datasets/home-data-for-ml-course/train.csv")
testdata1=testdata.copy()
testdata.drop(columns=["Id"],inplace=True)
traindata.drop(columns=["Id"],inplace=True)

In [3]:
traindata.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
y=traindata.SalePrice
X=traindata.drop("SalePrice",axis=1)

In [5]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [6]:
y.isnull().sum()

0

No null values exist in the target, which is noice :)

In [7]:
X=X.select_dtypes(exclude='object')  #I removed all columns which had didn't have numeric values for this tutorial.

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   OverallQual    1460 non-null   int64  
 4   OverallCond    1460 non-null   int64  
 5   YearBuilt      1460 non-null   int64  
 6   YearRemodAdd   1460 non-null   int64  
 7   MasVnrArea     1452 non-null   float64
 8   BsmtFinSF1     1460 non-null   int64  
 9   BsmtFinSF2     1460 non-null   int64  
 10  BsmtUnfSF      1460 non-null   int64  
 11  TotalBsmtSF    1460 non-null   int64  
 12  1stFlrSF       1460 non-null   int64  
 13  2ndFlrSF       1460 non-null   int64  
 14  LowQualFinSF   1460 non-null   int64  
 15  GrLivArea      1460 non-null   int64  
 16  BsmtFullBath   1460 non-null   int64  
 17  BsmtHalfBath   1460 non-null   int64  
 18  FullBath

Confirmed that only data types present are int and float

In [9]:
X_train, X_valid, y_train, y_valid=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

In [10]:
missing_vals=X_train.isnull().sum()
print(missing_vals[missing_vals>0])
#I wrote this so I can get Null values of each column

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [11]:
def tester(X_train,y_train,X_valid,y_valid):
    model=RandomForestRegressor(n_estimators=100,random_state=0)
    model.fit(X_train,y_train)
    predictions=model.predict(X_valid)
    mae=mean_absolute_error(y_valid,predictions)
    return mae

## Method 1: Dropping the columns which have NULL values entirely

In [12]:
#Lets create a duplicate of X_train and X_valid so we can compare the results later on

In [13]:
X_train_drop=X_train.copy()
X_valid_drop=X_valid.copy()
#The reason we had to write .copy() was because otherwise modifying the LHS would also modify the RHS. But now it won't modify the original dataset.
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
X_train_drop.drop(columns=cols_with_missing,inplace=True)
X_valid_drop.drop(columns=cols_with_missing,inplace=True)

Therefore, 3 columns which had missing values are removed successfully

In [14]:
mae=tester(X_train_drop,y_train,X_valid_drop,y_valid)
print(mae)

17837.82570776256


Note the mae value

## Method 2: Replace the NULL values with their means (IMPUTATION)

In [15]:
from sklearn.impute import SimpleImputer
my_imputer=SimpleImputer()
imputed_X_train=pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid=pd.DataFrame(my_imputer.transform(X_valid))

It has removed the column names, so we will add them back

In [16]:
imputed_X_train.columns=X_train.columns
imputed_X_valid.columns=X_valid.columns
imputed_X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20.0,90.000000,11694.0,9.0,5.0,2007.0,2007.0,452.0,48.0,0.0,...,774.0,0.0,108.0,0.0,0.0,260.0,0.0,0.0,7.0,2007.0
1,20.0,60.000000,6600.0,5.0,5.0,1962.0,1962.0,0.0,0.0,0.0,...,308.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2009.0
2,30.0,80.000000,13360.0,5.0,7.0,1921.0,2006.0,0.0,713.0,0.0,...,432.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,8.0,2009.0
3,20.0,69.614017,13265.0,8.0,5.0,2002.0,2002.0,148.0,1218.0,0.0,...,857.0,150.0,59.0,0.0,0.0,0.0,0.0,0.0,7.0,2008.0
4,20.0,118.000000,13704.0,7.0,5.0,2001.0,2002.0,150.0,0.0,0.0,...,843.0,468.0,81.0,0.0,0.0,0.0,0.0,0.0,1.0,2006.0


Nice !

In [17]:
mae1=tester(imputed_X_train,y_train,imputed_X_valid,y_valid)
print(mae1)

18062.894611872147


Look at how mae for dropping columns directly was infact lower

## Final Sol

I will take the direct dropping column approach as it was found more accurate

In [18]:
X_final_test=testdata
X_final_test=X_final_test.select_dtypes(exclude='object')

In [19]:
X_final_test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,506.0,0,82,0,0,144,0,0,1,2010


In [20]:
#X_final_train=X.drop(columns=cols_with_missing,axis=1)
#X_final_train.head()
my_imputer=SimpleImputer(strategy="median")
X_final_train=pd.DataFrame(my_imputer.fit_transform(X))
X_final_test=pd.DataFrame(my_imputer.fit_transform(X_final_test))

Note we now used median in simple imputer instead of mean, cuz we knew it would give better results. So yeah, there's that.

In [21]:
model_final=RandomForestRegressor(random_state=0)

In [22]:
model_final.fit(X_final_train,y)
final_predictions=model_final.predict(X_final_test)
final_predictions.shape

(1459,)

In [23]:
X_final_test.shape

(1459, 36)

In [24]:
testdata1["Id"]

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [25]:
output=pd.DataFrame({"Id": testdata1["Id"],"SalePrice": final_predictions})
output.to_csv('submission_house_kaggle_2.csv', index=False)